In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp cross

## Cross
> Using gamete operations to produce new individuals

In [ ]:
#| export

from chewc.core import *
from chewc.trait import *
from chewc.meiosis import *

import torch

In [ ]:
#| export
def x_random( genome: Genome, parent_haplotypes: torch.Tensor, n_crosses: int) -> torch.Tensor:
    """
    Generate random crosses from a set of parent haplotypes.

    Args:
    ----
        parent_haplotypes (torch.Tensor): Haplotypes of the parents. 
                                           Shape: (n_parents, ploidy, chr, loci)
        n_crosses (int): Number of crosses to generate.
        genome (Genome): Genome object.

    Returns:
    -------
        torch.Tensor: Haplotypes of the progeny. 
                      Shape: (n_crosses, ploidy, chr, loci)
    """
    
    assert len(parent_haplotypes.shape) == 4, f"Your input was {parent_haplotypes.shape} when it should be (#parents,ploidy,#chr,#loci)"
    device = genome.device
    n_parents = parent_haplotypes.shape[0]

    # Randomly select parents for each cross
    female_indices = torch.randint(0, n_parents, (n_crosses,), device=device)
    male_indices = torch.randint(0, n_parents, (n_crosses,), device=device)

    # Extract haplotypes of the selected parents
    female_haplotypes = parent_haplotypes[female_indices]
    male_haplotypes = parent_haplotypes[male_indices]

    # Simulate gametes
    female_gametes = simulate_gametes(genome, female_haplotypes)
    male_gametes = simulate_gametes(genome, male_haplotypes)

    # Combine gametes to form progeny haplotypes
    progeny_haplotypes = torch.cat([female_gametes, male_gametes], dim=1)

    return progeny_haplotypes

In [ ]:
ploidy = 2
n_chr = 5
n_loci = 1000
n_Ind = 333
g = Genome(ploidy, n_chr, n_loci)
population = Population()
population.create_random_founder_population(g, n_founders=n_Ind)
init_pop = population.get_genotypes().float()  # gets allele dosage for calculating trait values

# Convert to numpy array
boolean_array = x_random(g, init_pop, 10)

Created genetic map


In [ ]:
# %%timeit
# crossblock_random_crosses(g, init_pop, 1000)

In [ ]:
#| export
def x_DH(genome: Genome, parent_haplotypes: torch.Tensor, reps: int) -> torch.Tensor:
    """
    Generate doubled haploid individuals from a set of parent haplotypes with distinct samples for each repetition.

    Args:
    ----
        genome (Genome): Genome object.
        parent_haplotypes (torch.Tensor): Haplotypes of the parents. 
                                          Shape: (n_parents, ploidy, chr, loci)
        reps (int): Number of times to repeat the DH process with distinct samples.

    Returns:
    -------
        torch.Tensor: Haplotypes of the doubled haploid progeny. 
                      Shape: (n_parents, reps, ploidy, chr, loci)
    """
    ploidy, n_chr, n_loci = genome.shape()  # Assuming genome.shape() returns (ploidy, n_chr, n_loci)
    n_parents = parent_haplotypes.shape[0]
    # Each parent represents a new family created by this script
    all_dh_haplotypes_by_family = torch.zeros((n_parents, reps, ploidy, n_chr, n_loci), device=parent_haplotypes.device)
    
    # Need to loop through the reps/n_parents to fill the all_dh_haplotypes_by_family
    for rep in range(reps):
        gametes = simulate_gametes(genome, parent_haplotypes)  # Returns (n_parents, ploidy//2, n_chr, n_loci)
        # Create doubled haploid by copying the gametes into the ploidy axis
        doubled_haploids = torch.cat([gametes, gametes], dim=1)  # Concatenate gametes to double the haploid number
        all_dh_haplotypes_by_family[:, rep, :, :, :] = doubled_haploids
    
    return all_dh_haplotypes_by_family


In [ ]:
# %%timeit
x_DH(g, init_pop, 66)

tensor([[[[[1., 1., 1.,  ..., 1., 1., 1.],
           [0., 1., 1.,  ..., 0., 1., 1.],
           [0., 1., 1.,  ..., 0., 0., 1.],
           [1., 1., 1.,  ..., 1., 0., 1.],
           [0., 0., 1.,  ..., 0., 1., 1.]],

          [[1., 1., 1.,  ..., 1., 1., 1.],
           [0., 1., 1.,  ..., 0., 1., 1.],
           [0., 1., 1.,  ..., 0., 0., 1.],
           [1., 1., 1.,  ..., 1., 0., 1.],
           [0., 0., 1.,  ..., 0., 1., 1.]]],


         [[[1., 1., 1.,  ..., 1., 1., 1.],
           [0., 1., 1.,  ..., 0., 1., 1.],
           [0., 1., 1.,  ..., 0., 0., 1.],
           [1., 1., 1.,  ..., 1., 0., 1.],
           [0., 0., 1.,  ..., 0., 1., 1.]],

          [[1., 1., 1.,  ..., 1., 1., 1.],
           [0., 1., 1.,  ..., 0., 1., 1.],
           [0., 1., 1.,  ..., 0., 0., 1.],
           [1., 1., 1.,  ..., 1., 0., 1.],
           [0., 0., 1.,  ..., 0., 1., 1.]]],


         [[[1., 1., 1.,  ..., 1., 1., 1.],
           [0., 1., 1.,  ..., 0., 1., 1.],
           [0., 1., 1.,  ..., 0., 0., 1.],

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()